In [0]:
# Author: hzq
# Last review date: 2019/3/24

# Basic codes

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 
from google.colab import auth 
auth.authenticate_user() 
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p driver
!google-drive-ocamlfuse driver
import os

os.chdir("driver/Colab Notebooks/Santander_Cus_ Trans_kaggle/")
!ls

catboost_info		    generator.json	      requirements.txt
data			    generator_weights.hdf5    submit
discriminator.json	    infogan_hzq_20190319.csv  timeline
discriminator_weights.hdf5  lgbm_importances.png      train_loss_collector.csv
download		    LICENSE		      utils
FI_20190315.png		    model		      valid_loss_collector.csv
FI.png			    README.md


In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Main Parts

In [0]:
# if 'COLAB_TPU_ADDR' not in os.environ:
#   print('ERROR: Not connected to a TPU runtime')
# else:
#   tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
#   print ('TPU address is', tpu_address)
#   with tf.Session(tpu_address) as session:
#     devices = session.list_devices()
#   print('TPU devices:')
#   pprint.pprint(devices)

if 'COLAB_GPU' not in os.environ:
    print('ERROR: Not connected to a GPU runtime')
else: 
    print('GPU devices:')
    pprint.pprint(tf.test.gpu_device_name())
print('CPU devices:')
pprint.pprint(device_lib.list_local_devices())

In [0]:
# !cat /proc/meminfo
# !cat /proc/cpuinfo

## Import required packages

In [0]:
import random, copy, sys, gc, os  # sys.getsizeof()
import logging
import datetime
import warnings
import functools
import pprint

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames

from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.python.client import device_lib
warnings.filterwarnings('ignore')

In [0]:
import functools
from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, concatenate
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding1D, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling1D, Conv1D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf

Using TensorFlow backend.


## prepared process


### read data online

### read data offline

In [0]:
%%time
train_df = pd.read_csv("./data/data_raw/train/train.csv")
test_df = pd.read_csv("./data/data_raw/test/test.csv")

train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

idx = features = test_df.columns.values[1:202]
for df in [test_df, train_df]:
# for df in [test_df, X_train_add_df]:
    df['sum'] = df[idx].sum(axis=1)  
    df['min'] = df[idx].min(axis=1)
    df['max'] = df[idx].max(axis=1)
    df['mean'] = df[idx].mean(axis=1)
    df['std'] = df[idx].std(axis=1)
    df['skew'] = df[idx].skew(axis=1)
    df['kurt'] = df[idx].kurtosis(axis=1)
    df['med'] = df[idx].median(axis=1)

X_train_nullcount = train_df.isnull().mean()    
X_train_nullcount[X_train_nullcount!=0]
# X_train_add_nullcount = X_train_add_df.isnull().mean()    
# X_train_add_nullcount[X_train_add_nullcount!=0]


# Re-construct 2019/3/14: deal with the imbalance by over-sampling

# # !pip install imblearn
# from collections import Counter
# from imblearn.over_sampling import SMOTE 


# train_df_part = train_df.drop(["ID_code"], axis=1)
# X_train = train_df_part.drop(["target"], axis=1)
# y_train = train_df_part.loc[:, "target"]

# print('Original dataset shape {}'.format(Counter(y_train)))
# sm = SMOTE(random_state=42)
# X_train_add_arr, y_train_add_arr = sm.fit_sample(X_train, y_train)
# print('Resampled dataset shape {}'.format(Counter(y_train_add_arr)))

# X_train_add_df = pd.DataFrame(X_train_add_arr, columns=X_train.columns)
# y_train_add_df = pd.Series(y_train_add_arr)

# del train_df_part, X_train, y_train, X_train_add_arr, y_train_add_arr

Mem. usage decreased to 78.01 Mb (74.7% reduction)
Mem. usage decreased to 77.82 Mb (74.6% reduction)
CPU times: user 1min 6s, sys: 2.18 s, total: 1min 8s
Wall time: 1min 17s


In [0]:
# Note 2019/3/14: this is a work in progress, some of the features added here will be later dropped

# features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
# for feature in features:
#     train_df['r2_'+feature] = np.round(train_df[feature], 2)
#     test_df['r2_'+feature] = np.round(test_df[feature], 2)
#     train_df['r1_'+feature] = np.round(train_df[feature], 1)
#     test_df['r1_'+feature] = np.round(test_df[feature], 1)

In [0]:
# X_train_add_df.drop(["kurt"], axis=1,inplace=True) 
train_df.drop(["kurt"], axis=1,inplace=True) 

# features = [c for c in X_train_add_df.columns if c not in ['ID_code', 'target']]
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

# del train_df
# X_train_add_df = reduce_mem_usage(X_train_add_df)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df[features], target, test_size=0.1, shuffle=True)
X_test = test_df[features]

In [0]:
gc.collect()
sys._clear_type_cache()

###model: [InfoGan](https://github.com/eriklindernoren/Keras-GAN/blob/master/infogan/infogan.py)

#### constructs model

In [0]:
# TODO 2019/3/21: add docstrings following google style!

class INFOGAN():
    def __init__(self):
        self.img_rows = len(features)
        self.num_classes = 2
        self.img_shape = (self.img_rows, )
        self.latent_dim = 200
        self.loss_name = ['d_loss', 'q_loss', 'g_loss'] 
        optimizer = Adam(0.002, 0.5)
        losses = ['binary_crossentropy', self.mutual_info_loss]

        # Build and the discriminator and recognition network
        
        self.discriminator, self.auxilliary = self.build_disk_and_q_net()

        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the recognition network Q
        
        self.auxilliary.compile(loss=[self.mutual_info_loss],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        
        gen_input = Input(shape=(self.latent_dim, ))    
        img = self.generator(gen_input)

        # For the combined model we will only train the generator
        
        self.discriminator.trainable = False
        valid = self.discriminator(img)
        
        # The recognition network produces the label
        
        target_label = self.auxilliary(img)

        # The combined model  (stacked generator and discriminator)
        
        self.combined = Model(gen_input, [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)
        
    @functools.lru_cache(maxsize=128)
    def build_generator(self):
        model = Sequential()
        model.add(Dense(500, activation="relu", input_dim=self.latent_dim))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(300))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.img_rows))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("tanh"))
        gen_input = Input(shape=(self.latent_dim, ))
        img = model(gen_input)
        # print("The generator model structure is: ")
        # model.summary()
        return Model(gen_input, img)

    @functools.lru_cache(maxsize=128)
    def build_disk_and_q_net(self):

        img = Input(shape=self.img_shape)
        
        # Shared layers between discriminator and recognition network
        
        model = Sequential()
        model.add(Dense(300))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Dense(500))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(700))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(800))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        img_embedding = model(img)

        # Discriminator
        
        validity = Dense(1, activation='sigmoid')(img_embedding)
        # validity = Dense(1, activation='relu')(img_embedding)
        # print("The discriminator model structure is: ")        
        # model.summary()
        
        # Recognition
        
        q_net = Dense(128, activation='relu')(img_embedding)
        label = Dense(self.num_classes, activation='softmax')(q_net)
        # print("The recognition model structure is: ")        
        # model.summary()
        # Return discriminator and recognition network
        
        return Model(img, validity), Model(img, label)

    @functools.lru_cache(maxsize=128)
    def mutual_info_loss(self, c, c_given_x):
      
        """The mutual information metric we aim to minimize"""
        
        eps = 1e-8
        conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
        entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))
        return conditional_entropy + entropy
    
    @functools.lru_cache(maxsize=128)
    def sample_generator_input(self, batch_size):
        sampled_noise = np.random.normal(0, 1, (batch_size, self.latent_dim-self.num_classes))
        sampled_labels = np.random.randint(0, self.num_classes, batch_size).reshape(-1, 1)
        sampled_labels = to_categorical(sampled_labels, num_classes=self.num_classes)
        return sampled_noise, sampled_labels    
    
    @functools.lru_cache(maxsize=128)
    def train(self, epochs, batch_size=500):              
        # valid = np.ones((batch_size, ))  # Adversarial ground truths
        fake = np.zeros((batch_size, ))  # Adversarial ground truths
        self.loss_list_train = []
        self.loss_list_valid = []
        # acc, precision, recall, f1_score = 0, 0, 0, 0
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------          
            idx = np.random.choice(X_train.index, batch_size)
            idx_ = [np.where(np.array(X_train.index)==eachidx)[0][0] for eachidx in idx]
            idx_dict[epoch] = idx_
            imgs = X_train.iloc[idx_][:]
            valid_t = np.array([float(y_train.iloc[eachidx]) for eachidx in idx_])
            
            # Sample noise and categorical labels
            
            # labels_f = np.array([1.0 if label > 0.5 else 0 for label in np.random.uniform(0, 1, size=len(data_valid))])
            # labels_f = np.array([1.0 if random.uniform(0, 1) > 0.5 else 0.0 for indent in range(len(data_valid))])
            # sampled_noise, sampled_noise_valid, sampled_labels, sampled_labels_valid = train_test_split(data_valid, labels_f, train_size=batch_size, random_state=23)
            # sampled_noise, sampled_labels = sampled_noise.iloc[:, :self.latent_dim-2], to_categorical(sampled_labels.reshape(-1, 1), num_classes=self.num_classes)
            sampled_noise, sampled_labels = self.sample_generator_input(batch_size)
            gen_input = np.concatenate((sampled_noise, sampled_labels), axis=1)
            
            # Generate  new images
            
            gen_imgs = self.generator.predict(gen_input)
            
            # Train on real and generated data
            
            self.discriminator.trainable = True
            d_loss_real = self.discriminator.train_on_batch(imgs, valid_t)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)

            # Avg. loss
            
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator and Q-network
            # ---------------------
            self.discriminator.trainable = False
            g_loss = self.combined.train_on_batch(gen_input, [valid_t, sampled_labels]) 
            print ("Display train loss: %d [D loss: %.4f, acc.: %.4f%%] [Q loss: %.4f] [G loss: %.4f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[1], g_loss[2]))
            self.loss_list_train.append([d_loss[0], g_loss[1], g_loss[2]])

            # ---------------------
            #  Calculate validation loss
            # ---------------------           
            idx_valid = np.random.choice(X_valid.index, batch_size)
            idx_valid_ = [np.where(np.array(X_valid.index)==eachidx)[0][0] for eachidx in idx_valid]
            imgs_valid = X_valid.iloc[idx_valid_][:]
            valid_t_valid = np.array([float(y_valid.iloc[eachidx]) for eachidx in idx_valid_])
            
            # sampled_noise_valid, sampled_labels_valid = sampled_noise_valid.iloc[:batch_size, :self.latent_dim-2], to_categorical(sampled_labels_valid[:batch_size].reshape(-1, 1), num_classes=self.num_classes)
            sampled_noise_valid, sampled_labels_valid = self.sample_generator_input(batch_size)
            gen_input_valid = np.concatenate((sampled_noise_valid, sampled_labels_valid), axis=1)
            
            gen_imgs_valid = self.generator.predict(gen_input_valid)   
            
            d_loss_real_valid = self.discriminator.test_on_batch(imgs_valid, valid_t_valid)
            d_loss_fake_valid = self.discriminator.test_on_batch(gen_imgs_valid, fake)
            
            d_loss_valid = 0.5 * np.add(d_loss_real_valid, d_loss_fake_valid)
            
            g_loss_valid = self.combined.test_on_batch(gen_input_valid, [valid_t_valid, sampled_labels_valid])        
            # print ("Display valid loss: %d [D loss: %.4f, acc.: %.4f%%] [Q loss: %.4f] [G loss: %.4f]" % (epoch, d_loss_valid[0], 100*d_loss_valid[1], g_loss_valid[1], g_loss_valid[2]))
            self.loss_list_valid.append([d_loss_valid[0], g_loss_valid[1], g_loss_valid[2]])

            # acc, precision, recall, f1_score = \
                # acc + accuracy_score(valid_t_valid, self.labels_test_pred), \
                # precision + precision_score(valid_t_valid, self.labels_test_pred), \
                # recall + recall_score(valid_t_valid, self.labels_test_pred), \
               #  f1_score + f1_score(valid_t_valid, self.labels_test_pred)
    # print('Accuracy of the network on the ? validation data: %.3f %%' % (100*acc/epochs))
    # print('Precision of the network on the ? validation data: %.3f %%' % (100*precision/epochs))
    # print('Recall of the network on the ? validation data: %.3f %%' % (100*recall/epochs))
    # print('F1-score of the network on the ? validation data: %.3f %%' % (100*f1_score/epochs))

# TODO 2019/3/19: check the formation of prediction    
#     def predict(self):
#         return self.discriminator.predict(X_test)
        
    def save_model(self):
        def save(model, model_name):
            model_path = "%s.json" % model_name
            weights_path = "%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])
        save(self.generator, "generator")
        save(self.discriminator, "discriminator")
        # save(self.auxilliary, 'recognition')
      
    @functools.lru_cache(maxsize=128) 
    def save_loss(self):
        loss_df = pd.DataFrame(columns=list(self.loss_name), data=self.loss_list_train)
        loss_df.to_csv('train_loss_collector.csv', index=False)
        del  self.loss_list_train
#         loss_df = pd.DataFrame(columns=list(self.loss_name), data=self.loss_list_valid)
#         loss_df.to_csv('valid_loss_collector.csv', index=False)
#         del  self.loss_list_valid

#### evaluate model

In [0]:
batch_size = 500
epochs_num = 300
idx_dict = {}

with tf.device('/gpu:0'):
    X_train, X_valid, y_train, y_valid = \
        train_test_split(
                train_df[features], target, 
                train_size=int(0.8*(train_df.shape[0])), 
                random_state=23)
    infogan = INFOGAN()
    infogan.train(epochs=epochs_num, batch_size=batch_size)
    infogan.save_loss()
    infogan.save_model()

In [0]:
loss_train = pd.read_csv('train_loss_collector.csv')
loss_valid = pd.read_csv('valid_loss_collector.csv')

plt.plot()
plt.plot(np.arange(epochs_num), loss_train['d_loss'], 'r-', label='train')
plt.plot(np.arange(epochs_num), loss_valid['d_loss'], 'b-', label='valid')
plt.show()

In [0]:
loss_train.plot();
# loss_train.plot(kind='bar', stacked=True);
loss_data.div(loss_data.sum(axis=1), axis=0).plot(kind='barh', stacked=True)

In [0]:
loss_valid.plot();
# loss_valid.plot(kind='bar', stacked=True);

In [0]:
gc.collect()
sys._clear_type_cache()

#### submit prediction

In [0]:
y_predict = infogan.discriminator.predict(test_df[features])

submit = pd.DataFrame(test_df['ID_code'])
submit['target'] = y_predict

In [0]:
submit['target'].value_counts()

In [0]:
submit['target'][submit['target'] > 0.1]

In [0]:
submit.to_csv("infogan_hzq_20190319.csv", index=None)

###model: [TripleGan](https://github.com/taki0112/TripleGAN-Tensorflow)

#### constructs model

In [0]:
!pip install ops

Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-sgnc5shc/ops/


In [0]:
# import cifar10
from ops import *
from utils import *
import time

In [0]:
# TODO 2019/3/21: add docstrings following google style!
# TODO 2019/3/21: apply modification to use it in numerical classification task

class TripleGAN(object) :
    def __init__(self, sess, epoch, batch_size, unlabel_batch_size, z_dim, dataset_name, n, gan_lr, cla_lr, checkpoint_dir, result_dir, log_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.result_dir = result_dir
        self.log_dir = log_dir
        self.epoch = epoch
        self.batch_size = batch_size
        self.unlabelled_batch_size = unlabel_batch_size
        self.test_batch_size = 500
        self.model_name = "TripleGAN"  # name for checkpoint
        
        if self.dataset_name == 'cifar10' :
            self.input_height = 32
            self.input_width = 32
            self.output_height = 32
            self.output_width = 32

            self.z_dim = z_dim
            self.y_dim = 10
            self.c_dim = 3

            self.learning_rate = gan_lr # 3e-4, 1e-3
            self.cla_learning_rate = cla_lr # 3e-3, 1e-2 ?
            self.GAN_beta1 = 0.5
            self.beta1 = 0.9
            self.beta2 = 0.999
            self.epsilon = 1e-8
            self.alpha = 0.5
            self.alpha_cla_adv = 0.01
            self.init_alpha_p = 0.0 # 0.1, 0.03
            self.apply_alpha_p = 0.1
            self.apply_epoch = 200 # 200, 300
            self.decay_epoch = 50

            self.sample_num = 64
            self.visual_num = 100
            self.len_discrete_code = 10

            self.data_X, self.data_y, self.unlabelled_X, self.unlabelled_y, self.test_X, self.test_y = cifar10.prepare_data(n) # trainX, trainY, testX, testY

            self.num_batches = len(self.data_X) // self.batch_size

        else :
#             raise NotImplementedError
            self.input_height = 1
            self.input_width = len(features)
#             self.output_height = 32
#             self.output_width = 32

            self.z_dim = z_dim
            self.y_dim = 2
#             self.c_dim = 3

            self.learning_rate = gan_lr # 3e-4, 1e-3
            self.cla_learning_rate = cla_lr # 3e-3, 1e-2 ?
            self.GAN_beta1 = 0.5
            self.beta1 = 0.9
            self.beta2 = 0.999
            self.epsilon = 1e-8
            self.alpha = 0.5
            self.alpha_cla_adv = 0.01
            self.init_alpha_p = 0.0 # 0.1, 0.03
            self.apply_alpha_p = 0.1
            self.apply_epoch = 200 # 200, 300
            self.decay_epoch = 50

            self.sample_num = 64
            self.visual_num = 100
            self.len_discrete_code = 10

            self.data_X, self.data_y, self.unlabelled_X, self.unlabelled_y, self.test_X, self.test_y = X_train, y_train, X_valid, y_valid, X_test, y_test

            self.num_batches = len(self.data_X) // self.batch_size
            
    def discriminator(self, x, y_, scope='discriminator', is_training=True, reuse=False):
        with tf.variable_scope(scope, reuse=reuse) :
            x = dropout(x, rate=0.2, is_training=is_training)
            y = tf.reshape(y_, [-1, 1, 1, self.y_dim])
            x = conv_concat(x,y)

            x = lrelu(conv_layer(x, filter_size=32, kernel=[3,3], layer_name=scope+'_conv1'))
            x = conv_concat(x,y)
            x = lrelu(conv_layer(x, filter_size=32, kernel=[3,3], stride=2, layer_name=scope+'_conv2'))
            x = dropout(x, rate=0.2, is_training=is_training)
            x = conv_concat(x,y)

            x = lrelu(conv_layer(x, filter_size=64, kernel=[3,3], layer_name=scope+'_conv3'))
            x = conv_concat(x,y)
            x = lrelu(conv_layer(x, filter_size=64, kernel=[3,3], stride=2, layer_name=scope+'_conv4'))
            x = dropout(x, rate=0.2, is_training=is_training)
            x = conv_concat(x,y)

            x = lrelu(conv_layer(x, filter_size=128, kernel=[3,3], layer_name=scope+'_conv5'))
            x = conv_concat(x,y)
            x = lrelu(conv_layer(x, filter_size=128, kernel=[3,3], layer_name=scope+'_conv6'))
            x = conv_concat(x,y)

            x = Global_Average_Pooling(x)
            x = flatten(x)
            x = concat([x,y_]) # mlp_concat

            x_logit = linear(x, unit=1, layer_name=scope+'_linear1')
            out = sigmoid(x_logit)


            return out, x_logit, x

    def generator(self, z, y, scope='generator', is_training=True, reuse=False):
        with tf.variable_scope(scope, reuse=reuse) :

            x = concat([z, y]) # mlp_concat

            x = relu(linear(x, unit=512*4*4, layer_name=scope+'_linear1'))
            x = batch_norm(x, is_training=is_training, scope=scope+'_batch1')

            x = tf.reshape(x, shape=[-1, 4, 4, 512])
            y = tf.reshape(y, [-1, 1, 1, self.y_dim])
            x = conv_concat(x,y)

            x = relu(deconv_layer(x, filter_size=256, kernel=[5,5], stride=2, layer_name=scope+'_deconv1'))
            x = batch_norm(x, is_training=is_training, scope=scope+'_batch2')
            x = conv_concat(x,y)

            x = relu(deconv_layer(x, filter_size=128, kernel=[5,5], stride=2, layer_name=scope+'_deconv2'))
            x = batch_norm(x, is_training=is_training, scope=scope+'_batch3')
            x = conv_concat(x,y)

            x = tanh(deconv_layer(x, filter_size=3, kernel=[5,5], stride=2, wn=False, layer_name=scope+'deconv3'))

            return x
    def classifier(self, x, scope='classifier', is_training=True, reuse=False):
        with tf.variable_scope(scope, reuse=reuse) :
            x = gaussian_noise_layer(x) # default = 0.15
            x = lrelu(conv_layer(x, filter_size=128, kernel=[3,3], layer_name=scope+'_conv1'))
            x = lrelu(conv_layer(x, filter_size=128, kernel=[3,3], layer_name=scope+'_conv2'))
            x = lrelu(conv_layer(x, filter_size=128, kernel=[3,3], layer_name=scope+'_conv3'))

            x = max_pooling(x, kernel=[2,2], stride=2)
            x = dropout(x, rate=0.5, is_training=is_training)

            x = lrelu(conv_layer(x, filter_size=256, kernel=[3,3], layer_name=scope+'_conv4'))
            x = lrelu(conv_layer(x, filter_size=256, kernel=[3,3], layer_name=scope+'_conv5'))
            x = lrelu(conv_layer(x, filter_size=256, kernel=[3,3], layer_name=scope+'_conv6'))

            x = max_pooling(x, kernel=[2,2], stride=2)
            x = dropout(x, rate=0.5, is_training=is_training)

            x = lrelu(conv_layer(x, filter_size=512, kernel=[3,3], layer_name=scope+'_conv7'))
            x = nin(x, unit=256, layer_name=scope+'_nin1')
            x = nin(x, unit=128, layer_name=scope+'_nin2')

            x = Global_Average_Pooling(x)
            x = flatten(x)
            x = linear(x, unit=10, layer_name=scope+'_linear1')
            return x

    def build_model(self):
#         image_dims = [self.input_height, self.input_width, self.c_dim]
        image_dims = [self.input_height, self.input_width]
        bs = self.batch_size
        unlabel_bs = self.unlabelled_batch_size
        test_bs = self.test_batch_size
        alpha = self.alpha
        alpha_cla_adv = self.alpha_cla_adv
        self.alpha_p = tf.placeholder(tf.float32, name='alpha_p')
        self.gan_lr = tf.placeholder(tf.float32, name='gan_lr')
        self.cla_lr = tf.placeholder(tf.float32, name='cla_lr')
        self.unsup_weight = tf.placeholder(tf.float32, name='unsup_weight')
        self.c_beta1 = tf.placeholder(tf.float32, name='c_beta1')

        """ Graph Input """
        # images
        self.inputs = tf.placeholder(tf.float32, [bs] + image_dims, name='real_images')
        self.unlabelled_inputs = tf.placeholder(tf.float32, [unlabel_bs] + image_dims, name='unlabelled_images')
        self.test_inputs = tf.placeholder(tf.float32, [test_bs] + image_dims, name='test_images')

        # labels
        self.y = tf.placeholder(tf.float32, [bs, self.y_dim], name='y')
        self.unlabelled_inputs_y = tf.placeholder(tf.float32, [unlabel_bs, self.y_dim])
        self.test_label = tf.placeholder(tf.float32, [test_bs, self.y_dim], name='test_label')
        self.visual_y = tf.placeholder(tf.float32, [self.visual_num, self.y_dim], name='visual_y')

        # noises
        self.z = tf.placeholder(tf.float32, [bs, self.z_dim], name='z')
        self.visual_z = tf.placeholder(tf.float32, [self.visual_num, self.z_dim], name='visual_z')

        """ Loss Function """
        # A Game with Three Players

        # output of D for real images
        D_real, D_real_logits, _ = self.discriminator(self.inputs, self.y, is_training=True, reuse=False)

        # output of D for fake images
        G = self.generator(self.z, self.y, is_training=True, reuse=False)
        D_fake, D_fake_logits, _ = self.discriminator(G, self.y, is_training=True, reuse=True)

        # output of C for real images
        C_real_logits = self.classifier(self.inputs, is_training=True, reuse=False)
        R_L = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=C_real_logits))

        # output of D for unlabelled images
        Y_c = self.classifier(self.unlabelled_inputs, is_training=True, reuse=True)
        D_cla, D_cla_logits, _ = self.discriminator(self.unlabelled_inputs, Y_c, is_training=True, reuse=True)

        # output of C for fake images
        C_fake_logits = self.classifier(G, is_training=True, reuse=True)
        R_P = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=C_fake_logits))

        #

        # get loss for discriminator
        d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
        d_loss_fake = (1-alpha)*tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))
        d_loss_cla = alpha*tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_cla_logits, labels=tf.zeros_like(D_cla)))
        self.d_loss = d_loss_real + d_loss_fake + d_loss_cla

        # get loss for generator
        self.g_loss = (1-alpha)*tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake)))

        # test loss for classify
        test_Y = self.classifier(self.test_inputs, is_training=False, reuse=True)
        correct_prediction = tf.equal(tf.argmax(test_Y, 1), tf.argmax(self.test_label, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        # get loss for classify
        max_c = tf.cast(tf.argmax(Y_c, axis=1), tf.float32)
        c_loss_dis = tf.reduce_mean(max_c * tf.nn.softmax_cross_entropy_with_logits(logits=D_cla_logits, labels=tf.ones_like(D_cla)))
        # self.c_loss = alpha * c_loss_dis + R_L + self.alpha_p*R_P

        # R_UL = self.unsup_weight * tf.reduce_mean(tf.squared_difference(Y_c, self.unlabelled_inputs_y))
        self.c_loss = alpha_cla_adv * alpha * c_loss_dis + R_L + self.alpha_p*R_P

        """ Training """

        # divide trainable variables into a group for D and a group for G
        t_vars = tf.trainable_variables()
        d_vars = [var for var in t_vars if 'discriminator' in var.name]
        g_vars = [var for var in t_vars if 'generator' in var.name]
        c_vars = [var for var in t_vars if 'classifier' in var.name]

        for var in t_vars: print(var.name)
        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.d_optim = tf.train.AdamOptimizer(self.gan_lr, beta1=self.GAN_beta1).minimize(self.d_loss, var_list=d_vars)
            self.g_optim = tf.train.AdamOptimizer(self.gan_lr, beta1=self.GAN_beta1).minimize(self.g_loss, var_list=g_vars)
            self.c_optim = tf.train.AdamOptimizer(self.cla_lr, beta1=self.beta1, beta2=self.beta2, epsilon=self.epsilon).minimize(self.c_loss, var_list=c_vars)

        """" Testing """
        # for test
        self.fake_images = self.generator(self.visual_z, self.visual_y, is_training=False, reuse=True)

        """ Summary """
        d_loss_real_sum = tf.summary.scalar("d_loss_real", d_loss_real)
        d_loss_fake_sum = tf.summary.scalar("d_loss_fake", d_loss_fake)
        d_loss_cla_sum = tf.summary.scalar("d_loss_cla", d_loss_cla)

        d_loss_sum = tf.summary.scalar("d_loss", self.d_loss)
        g_loss_sum = tf.summary.scalar("g_loss", self.g_loss)
        c_loss_sum = tf.summary.scalar("c_loss", self.c_loss)



        # final summary operations
        self.g_sum = tf.summary.merge([d_loss_fake_sum, g_loss_sum])
        self.d_sum = tf.summary.merge([d_loss_real_sum, d_loss_sum])
        self.c_sum = tf.summary.merge([d_loss_cla_sum, c_loss_sum])


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
        gan_lr = self.learning_rate
        cla_lr = self.cla_learning_rate

        # graph inputs for visualize training results
        self.sample_z = np.random.uniform(-1, 1, size=(self.visual_num, self.z_dim))
        self.test_codes = self.data_y[0:self.visual_num]

        # saver to save model
        self.saver = tf.train.Saver()

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_counter = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_counter / self.num_batches)
            start_batch_id = checkpoint_counter - start_epoch * self.num_batches
            counter = checkpoint_counter
            with open('lr_logs.txt', 'r') as f :
                line = f.readlines()
                line = line[-1]
                gan_lr = float(line.split()[0])
                cla_lr = float(line.split()[1])
                print("gan_lr : ", gan_lr)
                print("cla_lr : ", cla_lr)
            print(" [*] Load SUCCESS")
        else:
            start_epoch = 0
            start_batch_id = 0
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()

        for epoch in range(start_epoch, self.epoch):

            if epoch >= self.decay_epoch :
                gan_lr *= 0.995
                cla_lr *= 0.99
                print("**** learning rate DECAY ****")
                print(gan_lr)
                print(cla_lr)

            if epoch >= self.apply_epoch :
                alpha_p = self.apply_alpha_p
            else :
                alpha_p = self.init_alpha_p

            rampup_value = rampup(epoch - 1)
            unsup_weight = rampup_value * 100.0 if epoch > 1 else 0

            # get batch data
            for idx in range(start_batch_id, self.num_batches):
                batch_images = self.data_X[idx * self.batch_size : (idx + 1) * self.batch_size]
                batch_codes = self.data_y[idx * self.batch_size : (idx + 1) * self.batch_size]

                batch_unlabelled_images = self.unlabelled_X[idx * self.unlabelled_batch_size : (idx + 1) * self.unlabelled_batch_size]
                batch_unlabelled_images_y = self.unlabelled_y[idx * self.unlabelled_batch_size : (idx + 1) * self.unlabelled_batch_size]

                batch_z = np.random.uniform(-1, 1, size=(self.batch_size, self.z_dim))

                feed_dict = {
                    self.inputs: batch_images, self.y: batch_codes,
                    self.unlabelled_inputs: batch_unlabelled_images,
                    self.unlabelled_inputs_y: batch_unlabelled_images_y,
                    self.z: batch_z, self.alpha_p: alpha_p,
                    self.gan_lr: gan_lr, self.cla_lr: cla_lr,
                    self.unsup_weight : unsup_weight
                }
                # update D network
                _, summary_str, d_loss = self.sess.run([self.d_optim, self.d_sum, self.d_loss], feed_dict=feed_dict)
                self.writer.add_summary(summary_str, counter)

                # update G network
                _, summary_str_g, g_loss = self.sess.run([self.g_optim, self.g_sum, self.g_loss], feed_dict=feed_dict)
                self.writer.add_summary(summary_str_g, counter)

                # update C network
                _, summary_str_c, c_loss = self.sess.run([self.c_optim, self.c_sum, self.c_loss], feed_dict=feed_dict)
                self.writer.add_summary(summary_str_c, counter)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: %.8f, g_loss: %.8f, c_loss: %.8f" \
                      % (epoch, idx, self.num_batches, time.time() - start_time, d_loss, g_loss, c_loss))

                # save training results for every 100 steps
                """
                if np.mod(counter, 100) == 0:
                    samples = self.sess.run(self.fake_images,
                                            feed_dict={self.z: self.sample_z, self.y: self.test_codes})
                    image_frame_dim = int(np.floor(np.sqrt(self.visual_num)))
                    save_images(samples[:image_frame_dim * image_frame_dim, :, :, :], [image_frame_dim, image_frame_dim],
                                './' + check_folder(
                                    self.result_dir + '/' + self.model_dir) + '/' + self.model_name + '_train_{:02d}_{:04d}.png'.format(
                                    epoch, idx))
                """

            # classifier test
            test_acc = 0.0

            for idx in range(10) :
                test_batch_x = self.test_X[idx * self.test_batch_size : (idx+1) * self.test_batch_size]
                test_batch_y = self.test_y[idx * self.test_batch_size : (idx+1) * self.test_batch_size]

                acc_ = self.sess.run(self.accuracy, feed_dict={
                    self.test_inputs: test_batch_x,
                    self.test_label: test_batch_y
                })

                test_acc += acc_
            test_acc /= 10

            summary_test = tf.Summary(value=[tf.Summary.Value(tag='test_accuracy', simple_value=test_acc)])
            self.writer.add_summary(summary_test, epoch)

            line = "Epoch: [%2d], test_acc: %.4f\n" % (epoch, test_acc)
            print(line)
            lr = "{} {}".format(gan_lr, cla_lr)
            with open('logs.txt', 'a') as f:
                f.write(line)
            with open('lr_logs.txt', 'a') as f :
                f.write(lr+'\n')

            # After an epoch, start_batch_id is set to zero
            # non-zero value is only for the first epoch after loading pre-trained model
            start_batch_id = 0

            # save model
            self.save(self.checkpoint_dir, counter)

            # show temporal results
            self.visualize_results(epoch)

            # save model for final step
        self.save(self.checkpoint_dir, counter)

    def visualize_results(self, epoch):
        # tot_num_samples = min(self.sample_num, self.batch_size)
        image_frame_dim = int(np.floor(np.sqrt(self.visual_num)))
        z_sample = np.random.uniform(-1, 1, size=(self.visual_num, self.z_dim))

        """ random noise, random discrete code, fixed continuous code """
        y = np.random.choice(self.len_discrete_code, self.visual_num)
        # Generated 10 labels with batch_size
        y_one_hot = np.zeros((self.visual_num, self.y_dim))
        y_one_hot[np.arange(self.visual_num), y] = 1

        samples = self.sess.run(self.fake_images, feed_dict={self.visual_z: z_sample, self.visual_y: y_one_hot})

        save_images(samples[:image_frame_dim * image_frame_dim, :, :, :], [image_frame_dim, image_frame_dim],
                    check_folder(
                        self.result_dir + '/' + self.model_dir + '/all_classes') + '/' + self.model_name + '_epoch%03d' % epoch + '_test_all_classes.png')

        """ specified condition, random noise """
        n_styles = 10  # must be less than or equal to self.batch_size

        np.random.seed()
        si = np.random.choice(self.visual_num, n_styles)

        for l in range(self.len_discrete_code):
            y = np.zeros(self.visual_num, dtype=np.int64) + l
            y_one_hot = np.zeros((self.visual_num, self.y_dim))
            y_one_hot[np.arange(self.visual_num), y] = 1

            samples = self.sess.run(self.fake_images, feed_dict={self.visual_z: z_sample, self.visual_y: y_one_hot})
            save_images(samples[:image_frame_dim * image_frame_dim, :, :, :], [image_frame_dim, image_frame_dim],
                        check_folder(
                            self.result_dir + '/' + self.model_dir + '/class_%d' % l) + '/' + self.model_name + '_epoch%03d' % epoch + '_test_class_%d.png' % l)

            samples = samples[si, :, :, :]

            if l == 0:
                all_samples = samples
            else:
                all_samples = np.concatenate((all_samples, samples), axis=0)

        """ save merged images to check style-consistency """
        canvas = np.zeros_like(all_samples)
        for s in range(n_styles):
            for c in range(self.len_discrete_code):
                canvas[s * self.len_discrete_code + c, :, :, :] = all_samples[c * n_styles + s, :, :, :]

        save_images(canvas, [n_styles, self.len_discrete_code],
                    check_folder(
                        self.result_dir + '/' + self.model_dir + '/all_classes_style_by_style') + '/' + self.model_name + '_epoch%03d' % epoch + '_test_all_classes_style_by_style.png')

    @property
    def model_dir(self):
        return "{}_{}_{}_{}".format(
            self.model_name, self.dataset_name,
            self.batch_size, self.z_dim)

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess, os.path.join(checkpoint_dir, self.model_name + '.model'), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            counter = int(next(re.finditer("(\d+)(?!.*\d)", ckpt_name)).group(0))
            print(" [*] Success to read {}".format(ckpt_name))
            return True, counter
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0

#### evaluate model

In [0]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    gan = TripleGAN(sess, epoch=args.epoch, batch_size=args.batch_size, unlabel_batch_size=args.unlabel_batch_size,
                        z_dim=args.z_dim, dataset_name=args.dataset, n=args.n, gan_lr = args.gan_lr, cla_lr = args.cla_lr,
                        checkpoint_dir=args.checkpoint_dir, result_dir=args.result_dir, log_dir=args.log_dir)

    # build graph
    gan.build_model()

    # show network architecture
    show_all_variables()

    # launch the graph in a session
    gan.train()
    print(" [*] Training finished!")

    # visualize learned generator
    gan.visualize_results(args.epoch-1)
    print(" [*] Testing finished!")

#### submit prediction